In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
import dataset as d
import model as m
import trainer2 as t
import util 
import eval as e
import loss as l
import config 
from importlib import reload


reload(config)
reload(m)
reload(d)
reload(l)
reload(util)
reload(t)
reload(e)

de = util.DataExtractor()
gt_df = de.croppedData()


dataset = d.CroppedDataset(gt_df, mode = "train")

model = m.YOLOResNet()
name = "wtf_is_goingon"
model = util.loadModel(name, model) #0.0122, 0.0059   #concorde0->concorde00

loader = DataLoader(dataset, batch_size=16, num_workers=12)
loss_fn = l.YOLOv2Loss(l_noobj = 0.10, l_wh = 1.0, l_obj = 4.0)
trainer = t.Trainer(model, loss_fn, gt_df, loader, epochs=12,  lr=1e-4, modelname=name, save_interval=4, device='cuda')

try:
    trainer.run()
except KeyboardInterrupt:
    print("Training interrupted. Saving model...")

util.saveModel(name, model)


Saving to: /workspace/mydir/obb_anns_hausarbeit/ds2_dense/ds2_dense/gt_space.json
[loadMode] Loaded weights from model_dumps/wtf_is_goingon.pth
2000 crops processed
0.03234643167771509
2000 crops processed
0.04058524032746028
2000 crops processed
0.04877234794370207
2000 crops processed
0.03369824144624282
2000 crops processed
0.03707673218587054
2000 crops processed
0.04083274530393496
Epoch 1 mAP:3.459798e-02total:95.6799 l_xy:9.8120 l_wh:42.8227 l_obj:1.9635 l_noobj:3.7060 l_cls:37.3758
2000 crops processed
0.045448861611804
2000 crops processed
0.061889548869882134
2000 crops processed
0.0431012127259363
2000 crops processed
0.05231140597291656
2000 crops processed
0.03612049669914694
2000 crops processed
0.0524152043816045
Epoch 2 mAP:3.165992e-02total:80.5123 l_xy:9.2501 l_wh:34.4128 l_obj:1.7968 l_noobj:3.3983 l_cls:31.6542
2000 crops processed
0.05506694767998717
2000 crops processed
0.05277196474392525
2000 crops processed
0.06602160499931543
2000 crops processed
0.06106501141

In [ ]:
import trainer2 as t2
import ModelSeries as ms
import loss as l
import util
import model as m
import config as c
import eval
import dataset
from importlib import reload
reload(dataset)
reload(c)
reload(t2)
reload(m)
reload(l)
reload(eval)
reload(ms)
reload(util)

gt_df = util.DataExtractor().croppedData()

modelseries = ms.ModelSeries("fix_obj_loss", gt_df=gt_df, model_descr="YoloResNet@120x40")
lc = ms.LearnConfig(c_lr=1e-3, c_obj = 5.0, c_noobj=0.1, c_xy = 1.0, c_wh = 1.0,  c_cls=1.0)


trainer = t2.Trainer(modelseries, learn_config = lc, epochs = 5, checkpoint_rate = 5, num_workers = 0, batch_size = 8)

try:
    trainer.run()
except KeyboardInterrupt:
    print("Training interrupted. Saving model...")
    print("training.ipynb - records.head()", trainer.modelseries.records.head())
    trainer.keyboardInterrupt()


Saving to: c:\Users\alexh\Desktop\cv3\obb_anns_hausarbeit\ds2_dense\ds2_dense\gt_space.json


In [10]:
reload(ms)
modelseries = ms.ModelSeries("fix_obj_loss", gt_df=gt_df, model_descr="YoloResNet@120x40")

ModelSeries: loadJsonData()


In [28]:
reload(ms)
series = ms.ModelSeries("debug_loss_ModelSeries_EvalRecord", gt_df=gt_df, model_descr="YoloResNet@120x40")

series.eval_records.df.head()



,class_id,class_name,n_occurences,n_images,epoch,mAP,mREC


In [6]:
train_data = (trainer.train_dataset.images, trainer.train_dataset.targets)
val_data = (trainer.eval_dataset.images, trainer.eval_dataset.targets)
trainer2 = t2.Trainer(modelseries, learn_config = lc, epochs = 1, checkpoint_rate = 5, num_workers = 0, batch_size = 8, val_gt = val_data, train_gt = train_data)

Saving to: c:\Users\alexh\Desktop\cv3\obb_anns_hausarbeit\ds2_dense\ds2_dense\gt_space.json


In [7]:
trainer2.run()

trainer.run(): Epoch 10


KeyboardInterrupt: 